# Titanic Survival Classification

## Table of Contents

1. [Exploratory Data Analysis](#heading1)    
    a. [Numerical Exploration](#subheading1)    
    b. [Visual Exploration](#subheading2)
    
2. [Preprocessing](#heading2)    
    a. [Extracting Formal Title from Passenger Name](#subheading3)    
    b. [Extracting Deck from Cabin Number](#subheading4)    
    c. [Missing Value Imputation](#subheading5)    
    d. [Categorical Variable Encoding](#subheading6)    
    e. [Feature Engineering](#subheading7)    
    f. [Feature Scaling](#subheading8)

3. [Modeling](#heading3)    
    a. [Logistic Regression](#subheading9)    
    b. [Random Forest](#subheading10)    
    c. [K-Nearest Neighbors](#subheading11)
    
4. [Generating Predictions](#heading4)
    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = 7, 5

In [ ]:
# Read in training data
titanic_data = pd.read_csv('../input/titanic/train.csv', index_col='PassengerId')

y = titanic_data['Survived']
X = titanic_data.drop('Survived', axis=1)

In [ ]:
# Read in test data
X_test = pd.read_csv('../input/titanic/test.csv', index_col='PassengerId')

# 1. Exploratory Data Analysis

## a. Numerical Exploration

In [ ]:
# Value counts for Survived (target variable)
y.value_counts()

In [ ]:
# Dimensions of training data (891 rows, 10 columns)
display(X.shape)

# Examine first five rows of training data
X.head()

In [ ]:
# Data types of variables (5 categorical, 5 numerical)
X.dtypes.value_counts()

In [ ]:
# Convert these numerical variables (Pclass, SibSp, Parch) to categorical
num_cols_to_cat = ['Pclass', 'SibSp', 'Parch']
X[num_cols_to_cat] = X[num_cols_to_cat].astype(str)
X_test[num_cols_to_cat] = X_test[num_cols_to_cat].astype(str)

In [ ]:
# Summary statistics for numerical variables
num_cols = [col for col in X.columns if X.dtypes[col] != 'object']
X[num_cols].describe()

In [ ]:
# Summary statistics for categorical variables
cat_cols = [col for col in X.columns if X.dtypes[col] == 'object']
X[cat_cols].describe()

## b. Visual Exploration

In [ ]:
# Count plot of Survived (target variable)
sns.countplot(x='Survived', data=titanic_data)
plt.title('Count Plot of Survived (Target Variable)');

In [ ]:
# Boxplot of Age (grouped by Survived)
sns.boxplot(x='Survived', y='Age', data=titanic_data)
plt.title('Boxplot of Age (Grouped by Survived)');

In [ ]:
# Boxplot of Fare (grouped by Survived)
sns.boxplot(x='Survived', y='Fare', data=titanic_data)
plt.title('Boxplot of Fare (Grouped by Survived)');

In [ ]:
# Histogram of Age
sns.distplot(titanic_data['Age'])
plt.title('Histogram of Age');

In [ ]:
# Histogram of Fare
sns.distplot(titanic_data['Fare'])
plt.title('Histogram of Fare');

In [ ]:
# Count plot of Pclass (grouped by Survived)
sns.countplot(x='Pclass', hue='Survived', data=titanic_data)
plt.title('Count Plot of Pclass (Grouped by Survived)')
plt.xlabel('Ticket Class');

In [ ]:
# Count plot of Sex (grouped by Survived)
sns.countplot(x='Sex', hue='Survived', data=titanic_data)
plt.title('Count Plot of Sex (Grouped by Survived)');

In [ ]:
# Count plot of SibSp (grouped by Survived)
sns.countplot(x='SibSp', hue='Survived', data=titanic_data)
plt.title('Count Plot of SibSp (Grouped by Survived)')
plt.xlabel('# of Siblings/Spouses')
plt.legend(title='Survived', loc='upper right');

In [ ]:
# Count plot of Parch (grouped by Survived)
sns.countplot(x='Parch', hue='Survived', data=titanic_data)
plt.title('Count Plot of Parch (Grouped by Survived)')
plt.xlabel('# of Parents/Children')
plt.legend(title='Survived', loc='upper right');

In [ ]:
# Count plot of Embarked (grouped by Survived)
sns.countplot(x='Embarked', hue='Survived', data=titanic_data)
plt.title('Count Plot of Embarked (Grouped by Survived)')
plt.xlabel('Port of Embarkation');

# 2. Preprocessing

## a. Extracting Formal Title from Passenger Name

We might be inclined to remove the Name variable since it is unique for each passenger. However, we see that the names contain formal titles such as "Mr" and "Mrs". The formal titles can contain useful information such as the age and gender of the passengers.

In [ ]:
# Extract title from passenger name
X['Title'] = X['Name'].str.extract(r',\s*([^.]*)\s*\.')
X_test['Title'] = X_test['Name'].str.extract(r',\s*([^.]*)\s*\.')

In [ ]:
# Value counts of title in training data
X['Title'].value_counts()

In [ ]:
# Value counts of title in test data
X_test['Title'].value_counts()

There are quite a few different titles, so let's combine some of the rare ones or consolidate them into the common categories. For example, professional titles such as "Dr" and "Rev" can be combined into a "Professional" category, while titles such as "Don" and "Sir" can simply be consolidated into the "Mr" category.

In [ ]:
def title_change(X):
    # Replace Don, Johkheer, and Sir with Mr
    X['Title'] = X['Title'].replace(['Don', 'Jonkheer', 'Sir'], 'Mr')

    # Replace Dr, Rev, Col, Major, and Capt with Professional
    X['Title'] = X['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Capt'], 'Professional')

    # Replace Mlle, Mme, Dona, Lady, and the Countess with Mrs
    X['Title'] = X['Title'].replace(['Mlle', 'Mme', 'Dona', 'Lady', 'the Countess'], 'Mrs')

    # Replace Ms with Miss
    X['Title'] = X['Title'].replace(['Ms'], 'Miss')

title_change(X)
title_change(X_test)

In [ ]:
# Value counts of title in training data (after title changes)
X['Title'].value_counts()

In [ ]:
# Value counts of title in test data (after title changes)
X_test['Title'].value_counts()

In [ ]:
# Remove passenger name now that we have their title
X = X.drop('Name', axis=1)
X_test = X_test.drop('Name', axis=1)

## b. Extracting Deck from Cabin Number

Like the Name variable, the Cabin variable can also contain useful information. In particular, the first character of each cabin number is a letter indicating the deck that the passenger resided on. 

In [ ]:
print('Number of NA in Cabin (training): ', X['Cabin'].isna().sum())
print('Number of NA in Cabin (test): ', X_test['Cabin'].isna().sum())

There are 687 and 327 missing values in the Cabin variable for the training and test sets, respectively. We can replace these missing values with "None", which will then allow us to extract the first character of each cabin number.

In [ ]:
# Replace missing cabin numbers with "None"
X['Cabin'] = X['Cabin'].fillna('None')
X_test['Cabin'] = X_test['Cabin'].fillna('None')

# Extract deck from the first character of cabin number
X['Deck'] = [cabin[0] for cabin in X['Cabin']]
X_test['Deck'] = [cabin[0] for cabin in X_test['Cabin']]

In [ ]:
# Remove cabin number now that we have deck information
X = X.drop('Cabin', axis=1)
X_test = X_test.drop('Cabin', axis=1)

In [ ]:
# Value counts of deck in training data
X['Deck'].value_counts()

In [ ]:
# Value counts of deck in test data
X_test['Deck'].value_counts()

## c. Missing Value Imputation

In [ ]:
# Number of missing values in each column of the training set (excluding columns with no missing values)
X.isna().sum()[X.isna().sum() > 0]

In [ ]:
# Number of missing values in each column of the test set (excluding columns with no missing values)
X_test.isna().sum()[X_test.isna().sum() > 0]

* `Age`: Impute NA with the median age corresponding to the passenger's ticket class (Pclass) and title. To avoid data leakage, we have to impute missing age in the test set with the median age (grouped by Pclass and title) from the training set. A way to do that is found [here](https://stackoverflow.com/questions/57699612/filling-in-missing-values-from-one-dataset-using-group-means-from-a-different-da).

In [ ]:
X['Age'] = X['Age'].fillna(X.groupby(['Pclass', 'Title'])['Age'].transform('median'))

# Create column for Age median grouped by Title
age_median = pd.DataFrame(X.groupby(['Pclass', 'Title'])['Age'].median()).rename(columns={'Age':'Age_median'})

# Merge Age median with test set
X_test_tmp = X_test.reset_index().merge(age_median, on=['Pclass', 'Title']).set_index('PassengerId')

# Fill missing Age in test set with Age median
X_test_tmp['Age'] = X_test_tmp['Age'].fillna(X_test_tmp['Age_median'])

# Drop Age median
X_test = X_test_tmp.drop('Age_median', axis=1)

* `Embarked`: Impute NA with the most frequent value.

In [ ]:
X['Embarked'] = X['Embarked'].fillna(X['Embarked'].value_counts().idxmax())
X_test['Embarked'] = X_test['Embarked'].fillna(X['Embarked'].value_counts().idxmax())

* `Fare`: Impute NA with the median.

In [ ]:
X['Fare'] = X['Fare'].fillna(X['Fare'].median())
X_test['Fare'] = X_test['Fare'].fillna(X['Fare'].median())

* `Ticket`: Drop this column for now.

In [ ]:
# Drop Ticket
X = X.drop('Ticket', axis=1)
X_test = X_test.drop('Ticket', axis=1)

## d. Categorical Variable Encoding

In [ ]:
# Convert these categorical variables (Pclass, SibSp, Parch) to numerical since they are ordinal in nature
cat_cols_to_num = ['Pclass', 'SibSp', 'Parch']
X[cat_cols_to_num] = X[cat_cols_to_num].astype(int)
X_test[cat_cols_to_num] = X_test[cat_cols_to_num].astype(int)

In [ ]:
# Ordinal encode Deck
# def ordinal_encoder(X):
#     X = X.replace({
#         'Deck': {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'T':8, 'N':9}
#     })

# ordinal_encoder(X)
# ordinal_encoder(X_test)

In [ ]:
# Apply one-hot encoding to the remaining categorical variables
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

In [ ]:
# Training set now has more columns than test set due to dummy variables only found in training set
print(X.shape)
print(X_test.shape)

In [ ]:
# Find columns that are in training set but not test set
missing_cols = set(X.columns) - set(X_test.columns)

# Add these columns to test set filled with 0's
for col in missing_cols:
    X_test[col] = 0
    
# Align columns of training and test sets
X_test = X_test[X.columns]

print(X.shape)
print(X_test.shape)

## e. Feature Engineering

Let's create some new features that can potentially improve predictive power (datatypes in parentheses):

* `Relatives` (int): Total number of relatives (siblings/spouses + parents/children)

* `HasRelatives` (bool): Does the passenger have relatives on board?

In [ ]:
def create_features(X):
    # Total number of relatives (siblings/spouses + parents/children)
    X['Relatives'] = X['SibSp'] + X['Parch']
    
    # Does the passenger have relatives on board?
    X['HasRelatives'] = X['Relatives'] > 0
    
create_features(X)
create_features(X_test)

## f. Feature Scaling

In [ ]:
# Feature scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Scaling removes column names and index; get them back
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

# 3. Modeling

In [ ]:
# Fuction to calculate CV accuracy
def CV_accuracy(estimator, X, y, cv):
    accuracy = cross_val_score(estimator=estimator, X=X, y=y, cv=cv, scoring='accuracy')
    return accuracy

## a. Logistic Regression

In [ ]:
# # Hyperparameter tuning
# param_grid = {'penalty': ['l1', 'l2'],
#               'C': [0.001, 0.01, 0.1, 1, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80]}

# logistic_model = GridSearchCV(estimator=LogisticRegression(solver='liblinear', random_state=1), param_grid=param_grid, cv=5, scoring='accuracy')
# logistic_model.fit(X_scaled, y)

# print(logistic_model.best_params_)
# print(logistic_model.best_score_)

In [ ]:
logistic_model = LogisticRegression(C=1, penalty='l2', solver='liblinear', random_state=1)
logistic_model.fit(X_scaled, y)

accuracy = CV_accuracy(estimator=logistic_model, X=X_scaled, y=y, cv=10)
print('----Logistic Regression----')
print('CV Accuracy: {:.4f}'.format(accuracy.mean()))
print('Std Deviation: {:.4f}'.format(accuracy.std()))

In [ ]:
# Logistic test predictions 
logistic_pred = logistic_model.predict(X_test_scaled)

## b. Random Forest

In [ ]:
# # Hyperparameter tuning
# param_grid = {'n_estimators': [200, 300, 400, 500],
#               'max_depth': range(6, 12),
#               'max_features': ['auto', 'sqrt', 'log2'],
#               'criterion': ['gini', 'entropy'],
#               'min_samples_split': range(2, 10),
#               'min_samples_leaf': range(2, 10)}

# rf_model = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=1), param_distributions=param_grid, cv=5, scoring='accuracy', random_state=1)
# rf_model.fit(X, y)

# print(rf_model.best_params_)
# print(rf_model.best_score_)

In [ ]:
rf_model = RandomForestClassifier(max_depth=9, max_features='log2', criterion='gini', min_samples_split=5, 
                                  min_samples_leaf=4, n_estimators=500, random_state=1)
rf_model.fit(X_scaled, y)

accuracy = CV_accuracy(estimator=rf_model, X=X_scaled, y=y, cv=10)
print('----Random Forest (tuned)----')
print('CV Accuracy: {:.4f}'.format(accuracy.mean()))
print('Std Deviation: {:.4f}'.format(accuracy.std()))

In [ ]:
# Random forest test predictions 
rf_pred = rf_model.predict(X_test_scaled)

## c. K-Nearest Neighbors

In [ ]:
# # Hyperparameter tuning
# param_grid = {'n_neighbors': range(3, 20, 2),
#               'weights': ['uniform', 'distance'],
#               'metric': ['euclidean', 'manhattan']}

# knn_model = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')
# knn_model.fit(X_scaled, y)

# print(knn_model.best_params_)
# print(knn_model.best_score_)

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=13, weights='uniform', metric='manhattan')
knn_model.fit(X_scaled, y)

accuracy = CV_accuracy(estimator=knn_model, X=X_scaled, y=y, cv=10)
print('----K-Nearest Neighbors----')
print('CV Accuracy: {:.4f}'.format(accuracy.mean()))
print('Std Deviation: {:.4f}'.format(accuracy.std()))

In [ ]:
# KNN test predictions 
knn_pred = knn_model.predict(X_test_scaled)

## Voting Classifier

We can combine the predictions of the models through voting. We will use soft voting, which labels the test set based on the average of the predicted probabilities of the models. Hard voting, which labels the test set based on the class voted by a majority of the models, can be used as well.

In [ ]:
voting_model = VotingClassifier(estimators=[('logistic', logistic_model), ('rf', rf_model), ('knn', knn_model)], voting='soft')
voting_model.fit(X_scaled, y)

accuracy = CV_accuracy(estimator=voting_model, X=X_scaled, y=y, cv=10)
print('----Voting Classifier----')
print('CV Accuracy: {:.4f}'.format(accuracy.mean()))
print('Std Deviation: {:.4f}'.format(accuracy.std()))

In [ ]:
# Voting test predictions 
voting_pred = voting_model.predict(X_test_scaled)

# 4. Generating Predictions

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'PassengerId': X_test.index,
                        'Survived': voting_pred})
output.to_csv('submission.csv', index=False)